# Inference notebook

In [1]:
import sys

sys.path.append("..")
sys.path.append("../../inputs")

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
from omegaconf import OmegaConf

c = OmegaConf.load("../config/main.yaml")

c.settings.debug = False
c.wandb.enabled = False
c.settings.dirs.working = ".."
c.settings.dirs.input = "../../inputs/"

pretrained = """
- dir: ../../datasets/trainings/2022-01-26_07-47-01/fold0/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-03/fold1/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-05/fold2/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-07/fold3/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-10/fold4/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-12/fold5/
  model: ump_1
- dir: ../../datasets/trainings/2022-01-26_07-47-14/fold6/
  model: ump_1
"""

c.params.pretrained = OmegaConf.create(pretrained)

log.info(OmegaConf.to_yaml(c))

2022-01-26 20:29:27,266 [INFO] [561483431] defaults:
- _self_
hydra:
  run:
    dir: ../outputs/${now:%Y-%m-%d_%H-%M-%S}
  job_logging:
    formatters:
      simple:
        format: '%(asctime)s [%(levelname)s][%(module)s] %(message)s'
wandb:
  enabled: false
  entity: imokuri
  project: ump
  dir: ${hydra:runtime.cwd}/../cache
  group: default
settings:
  print_freq: 100
  gpus: 6,7
  dirs:
    working: ..
    input: ../../inputs/
  inputs:
  - train.csv
  - example_test.csv
  - example_sample_submission.csv
  debug: false
  n_debug_data: 100000
  amp: true
  multi_gpu: true
params:
  seed: 440
  n_class: 1
  n_fold: 7
  skip_training: false
  epoch: 20
  es_patience: 0
  batch_size: 640
  gradient_acc_step: 1
  max_grad_norm: 1000
  fold: group
  group_name: investment_id
  label_name: target
  dataset: ump_1
  model: ump_1
  pretrained:
  - dir: ../../datasets/trainings/2022-01-26_07-47-01/fold0/
    model: ump_1
  - dir: ../../datasets/trainings/2022-01-26_07-47-03/fold1/
    model

In [4]:
import src.utils as utils
import ubiquant
from src.load_model import load_model
from src.run_loop import inference

In [5]:
utils.fix_seed(c.params.seed)
device = utils.gpu_settings(c)

2022-01-26 20:29:28,617 [INFO] [utils] Fix seed: 440
2022-01-26 20:29:28,619 [INFO] [utils] CUDA_VISIBLE_DEVICES: 6,7
2022-01-26 20:29:28,715 [INFO] [utils] torch device: cuda, device count: 2


In [6]:
env = ubiquant.make_env()  # initialize the environment
iter_test = env.iter_test()  # an iterator which loops over the test set and sample submission

models = load_model(c, device)

In [7]:
for test_df, sample_prediction_df in iter_test:

    df = inference(c, test_df, device, models)
    
    sample_prediction_df["target"] = df["target"]  # make your predictions here
    env.predict(sample_prediction_df)  # register your predictions

0it [00:00, ?it/s]

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
